In [ ]:
def evaluate4(file, logs, method, threshold, dataset):#-------------------EVALUATE4
    groupsDictOut = []
    groupsDictOut.append(["Super_Word", "Value"])
    wordsSeen = {}
    groups = {}
    itemsSeen = {}
    #var dictionary
    #file                                           #raw input file
    #start                                          #starting point for intervals
    #end                                            #end point for current interval
    count = Counter()                               #counter for frequencies
    timeSlice = 1                                   #timeSlice ID
    newIdea = 'false'                               #true/false:
                                                        #new bin in the time slice?
    newIdeaCount = 0                                #number of new ideas so far
    totalIdeas = 0                                  #total ideas per timeSlice
    output = []                                     #output array
    i = 0                                           #index for while loop
    model = models.Word2Vec.load("text8Model")
    #file.sort(key=itemgetter(3))
    
    logs.append(["--------------- START EXECUTION ---------------"])
    if(INTERVAL_MODE == 'time'):
        start = int(file[0][3])                     #unix time of first row
        end = start + INTERVAL
    elif(INTERVAL_MODE == 'count'):
        start = 0
        end = start + INTERVAL
        
    #output.append(["Time Slice",                        #file header
    #            "New Category?", 
    #            "# of New Categories", 
    #            "Total Ideas in Time Slice",
    #            "Probability of New Bin",
    #            "%New Categories in Time Slice",
    #            "%New Categories Overall",
    #            "Counter"])
    logs.append([str(timeSlice) + "------ TIME_SLICE" + str(timeSlice) + " -----"])
    while(i < len(file)):                                   #while more ideas still exist
        line = file[i]                                      #a line in a file
        if(INTERVAL_MODE == 'time'):
            current = int(line[3])
        elif(INTERVAL_MODE == 'count'):
            current = i          
        if(current <= end):                            #for each time slice
            format(line)                                    #convert idea content to list of relivant words
            logs.append(["\tIDEA " + str(i) + " -----"])
            # print groupType
            groupList = []
            for word in line[1]:                            #add to counter
                if(method == "nltk"):
                    group = getGroup(count, word, threshold, wordsSeen, groups)
                elif(method == "word2vec"):
                    group = getGroup3(count, word, model, threshold, wordsSeen, groups)
                oldCount = count[group]
                count[group] += 1
                newCount = count[group]
                #groupList.append(group)

                if newCount == 1:                           #was this a new bin?
                    newIdea = 'true'                            #if yes,
                    newIdeaCount += 1
                totalIdeas += 1
                logs.append(["\t\t" + word + ":\t" + group + "--> \t" + str(newCount)])
            for group in groupList:
                print group
            
            #COMBOS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            #groupList = getCombos(groupList)
            #for combo in groupList:
            #    oldCount = count[combo]
            #    count[combo] += 1
            #    newCount = count[combo]
            #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
            #    if(newCount == 1 and newIdea == "false"):
            #        newIdeaCount += 1
            #    logs.append(["\t\t" + str(line[1]) + ":\t" + combo + "--> \t" + str(newCount)])
            i += 1
        else:                                                   #at the end of each time slice
            if(totalIdeas > 0):
                output.append([timeSlice, newIdea, newIdeaCount, totalIdeas])
                estimateNewIdea(count, output[timeSlice - 1])
                calculateCat(count, output, timeSlice - 1, newIdeaCount, totalIdeas)
                timeSlice += 1                                  #increment time slice id
                logs.append([str(timeSlice) + "------ TIME_SLICE" + str(timeSlice) + " -----"])
                
            start = end                                         #update time slice markers
            end = start + INTERVAL
            newIdea = 'false'
            newIdeaCount = 0
            totalIdeas = 0
    if(totalIdeas > 0):
        output.append([timeSlice, newIdea, newIdeaCount, totalIdeas])
        estimateNewIdea(count, output[timeSlice -1])
        calculateCat(count, output, timeSlice - 1, newIdeaCount, totalIdeas)
    #print count
    dictionary_to_save = "Dictionaries/wordkeyMyData_%s_%s_%.2f" %(dataset, method, threshold)
    dictionary_out = open(dictionary_to_save, 'w')
    dictionary_out.write(json.dumps(wordsSeen, indent=2))
    dictionary_out.close()
    for key in groups.keys():
        groupsDictOut.append([key, groups.get(key)])
    
    
    for line in groupsDictOut:
        print line
    print count
    writeOut(groupsDictOut, "Dictionaries/groupkeyMyData_%s_%s_%.2f.csv" %(dataset, method, threshold))
    return output





def evaluate6(file, logs):#-------------------------------------------------EVALUATE6
    #datasets = ['SuperBoring',
    #             'Boring',
    #             'Normal',
    #             'NewAtEnd',
    #             'Exciting']              #the list of input filenames
    datasets = ['smallIdeas']
    # versions = [evaluate4(file, logs, "getGroup", 0.5)]       #array of functions to try out
    #             evaluate4(file, logs, "getGroup", 0.9),
    #             evaluate4(file, logs, "getGroup3", 0.5),
    #             evaluate4(file, logs, "getGroup3", 0.9)]
    #methods = ["nltk", "word2vec"]
    methods = ["word2vec"]
    # method_names = {"getGroup": 'nltk', 'getGroup3': 'word2vec'}
    thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    #thresholds = [0.6]
    vText = ['nltk 0.5','nltk 0.9',                 #strings associated with the function of 
             'word2vec 0.5','word2vec 0.9']                       
                                                    #"versions[]"
    output = []                                     #output to be sent to csv
    outLine = []                                    #one line of output taken from "returnedOut"
    # i = 0

    output.append(["dataset", "method",             #header
               "timeSlice", "GT_predict", "true"])
    
    

    for dataset in datasets:                     #for each dataset
        for theme, theme_data in dataset.groupby("theme"):
            input_file = "Input/%s.csv" %dataset
            print "datasets: ", dataset
            print "theme: ", theme
            # j = 0                   
            # for function in versions:                       #run each version of the program
            for method in methods:
                print "method: ", method
                for threshold in thresholds:
                    print "threshold: ", threshold
                    opened_input = getInputFile(input_file)
                    returnedOut = evaluate4(opened_input, logs, method, threshold, dataset)                      #output from this version
                    print "\treturnedOut: ", returnedOut
                    for line in returnedOut:                    #rotate output to desired format
                        print "\t\tline: ", line                    #and save to "output[]"
                        outLine = []
                        outLine.append(dataset)
                        outLine.append("%s_%.2f" %(method, threshold))
                        outLine.append(line[0])
                        outLine.append(line[4])
                        outLine.append(line[5])             
                        print "\t\t\t", outLine
                        output.append(outLine) 
            # j = j+1                       
        # i = i+1
    writeOut(output, OUTPUT_FILE)

def errorLog(errorCode):#---------------------------------------------ERROR_LOG
    if(errorCode == 0):
        return "ERROR: NO VERSION SPECIFIED"
    else:
        return "THERE WAS AN ERROR"
        
    
def writeOut(output, fileName):#--------------------------------------WRITE_OUT
    #var dictionary
    #filename                                       #file to write to
    #output                                         #data to write
    #writer                                         #csv writer object
    
    with open(fileName, 'w') as file:               
        writer = csv.writer(file)
        for row in output:
            writer.writerow(row)
        if(output == logs):
            print "Log file written to ", fileName
        else:
            print "Results written to ", fileName
            logs.append(["Results written to " + fileName])

#--------------------------------------------------------------------------MAIN

INPUT_FILE = "Input/Normal.csv"            
OUTPUT_FILE = "Data Output/MyDataWord2VecWithStemming.csv"
LOG_FILE = "MyDataWord2VecWithStemming.txt"
VERSION = 6
INTERVAL_MODE = 'count'                              #options: time, count;
                                                    #options: words, categories;
#INTERVAL = 60000                                    #1 minute
#INTERVAL = 600000                                    #10 minutes
#INTERVAL = 1800000                                  #30 minutes
#INTERVAL = 3600000                                  # 1 hour
INTERVAL = 3
out = []                                            #data output to print
logs = []                                           #log  output to print
#f = open('dictNLTK.file', 'r')
#word2Vec = eval(f.read())                          #dictionary in key-value form where
                                                        #key = words and their synonymns
                                                        #value = the group that is entered
                                                            #the counter for that word
#f.close()

#f = open('dict.file', 'r')
#nltk = eval(f.read())
#f.close()

#wordsSeen = {}
#itemsSeen = {}

#getInput
file = getInputFile(INPUT_FILE)

#setup Log File
logs.append(["VERSION:\t" + str(VERSION)])
logs.append(["INPUT_FILE:\t" + INPUT_FILE])
logs.append(["INTERVAL_MODE:\t" + INTERVAL_MODE])

#process
if(VERSION == 1):
    out = evaluate(file)
elif(VERSION == 2):
    out = evaluate2(file)
elif(VERSION == 3):
    out = evaluate3(file)
elif(VERSION == 4):
    out = evaluate4(file, logs)
elif(VERSION == 5):
    out = evaluate5(file, logs)
elif(VERSION == 6):
    evaluate6(file, logs)
else:
    print "NO VERSION SPECIFIED"
    logs.append([errorLog(0)])
    writeOut(logs, LOG_FILE)
    quit()

#print wordsSeen
#countUniqueWords(file)

#print output to screen
#f = open('dictNLTK.file', 'w')
#f.write(str(word2Vec))
#f.close()

#f = open('dict.file', 'w')
#f.write(str(nltk))
#f.close()

#for line in out:
#   print line

#VERSION 1:     basic; category as bin              IN-PROGRESS -- functional
#               INPUTS:
#                   ideas.csv
#               TODO:
#                   - implement log
#VERSION 2:     words as bins                       IN-PROGRESS -- functional   
#               INPUTS:
#                   ideas2.csv
#               TODO:
#                   - implement log
#VERSION 3:     words as bins; separate by theme    IN-PROGRESS -- not functional
#               INPUTS:
#                   ideas_corrected.csv
#                   smallIdeas.csv
#               TODO:
#                   - fix bugs that arose in compilation
#                   - implement log
#VERSION 4:     super-words as bins                 IN-PROGRESS -- functional
#               INPUTS:
#                   ideas_corrected.csv
#                   smallIdeas.csv
#               TODO:
#                   improve time/accuracy 
#VERSION 5:     super-words as bins;                IN-PROGRESS -- functional
#               separate by theme
#               TODO:
#                   - flip data 
#VERSION 6:     full test                           IN-PROGRESS -- not functional

#printResults
#writeOut(out, OUTPUT_FILE)
writeOut(logs, LOG_FILE)